In [1]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
import sys
sys.path.append('..')

import jax
import jax.numpy as jnp
import numpy as np
import os
import random

from tools.simulation import setup_event_simulator, normalize
from tools.generate import generate_multi_folder_events
from tools.utils import load_single_event, save_single_event
from tqdm import tqdm 

from tools.utils import read_photon_data_from_root, read_multi_folder_events

# Set up simulator in data mode
event_simulator = setup_event_simulator(
    json_filename='../config/cyl_geom_config.json',
    temperature=0., # no photon relaxation for data-like events
    K=2,
    is_data=True,
    max_detectors_per_cell=4
)

In [4]:
# Generate events in multiple folders
folders = ["../datasets/events_1ring", "../datasets/events_2rings", "../datasets/events_3rings"]
events_per_folder = 10  # 50 events in each folder
n_rings_list = [1, 2, 3]  # Different number of rings for each folder

# Define the file paths
muon_file = '/sdf/home/c/cjesus/GenWCSim/CProfProd/muon/0/cprof.root'
pion_file = '/sdf/home/c/cjesus/GenWCSim/CProfProd/pion/0/cprof.root'

# Example use of the multi-folder generation
results = generate_multi_folder_events(
    event_simulator=event_simulator,
    root_file_path=muon_file,
    folder_names=folders,
    events_per_folder=events_per_folder,
    n_rings_list=n_rings_list,
    pion_root_file_path=pion_file,
    batch_size=25
)


--------------------------------------------------------------------------------
Processing folder 1/3: ../datasets/events_1ring
Generating 10 events with 1 ring(s)
--------------------------------------------------------------------------------

Processing 10 events with 1 ring (muon)...
Using batch size of 25 events for multithreaded I/O
Saving events to directory: ../datasets/events_1ring
Processing batch 1/1 (events 0 to 9)


Saving batch 1: 100%|██████████| 10/10 [00:00<00:00, 126.37file/s]


Successfully processed 10 events.
All events saved to ../datasets/events_1ring with sequential naming (event_0.h5, event_1.h5, ...)

--------------------------------------------------------------------------------
Processing folder 2/3: ../datasets/events_2rings
Generating 10 events with 2 ring(s)
--------------------------------------------------------------------------------

Processing 10 events with 2 rings (muon + 1 pions)...
Using batch size of 25 events for multithreaded I/O
Saving events to directory: ../datasets/events_2rings
Processing batch 1/1 (events 0 to 9)


Saving batch 1: 100%|██████████| 10/10 [00:00<00:00, 12.35file/s]


Successfully processed 10 events.
All events saved to ../datasets/events_2rings with sequential naming (event_0.h5, event_1.h5, ...)

--------------------------------------------------------------------------------
Processing folder 3/3: ../datasets/events_3rings
Generating 10 events with 3 ring(s)
--------------------------------------------------------------------------------

Processing 10 events with 3 rings (muon + 2 pions)...
Using batch size of 25 events for multithreaded I/O
Saving events to directory: ../datasets/events_3rings
Processing batch 1/1 (events 0 to 9)


Saving batch 1: 100%|██████████| 10/10 [00:01<00:00,  6.61file/s]

Successfully processed 10 events.
All events saved to ../datasets/events_3rings with sequential naming (event_0.h5, event_1.h5, ...)

Generation Summary:
Total events generated: 30
  - ../datasets/events_1ring: 10 events
  - ../datasets/events_2rings: 10 events
  - ../datasets/events_3rings: 10 events


In [5]:
# Read events from the generated folders
folder_data = read_multi_folder_events(
    folder_names=folders,
    max_files_per_folder=10,  # Read only first 10 files from each folder
    summary_only=True
)


Reading events from 3 folders:

--------------------------------------------------
Folder 1/3: ../datasets/events_1ring
--------------------------------------------------
Found 10 files to analyze

Summary Statistics for 10 Events
Total number of tracks: 10
Total muons: 10 (100.0%)
Total pions: 0 (0.0%)

Charge Statistics:
Mean charge per track: 13272.58
Min charge: 1743.34
Max charge: 24871.82

Momentum Statistics:
Mean momentum magnitude: 740.43 MeV/c
Min momentum: 293.57 MeV/c
Max momentum: 1084.96 MeV/c

PMT Statistics Across All Events:
Number of active PMTs: 8917 / 9946
Mean charge per active PMT: 14.88

--------------------------------------------------
Folder 2/3: ../datasets/events_2rings
--------------------------------------------------
Found 10 files to analyze

Summary Statistics for 10 Events
Total number of tracks: 20
Total muons: 10 (50.0%)
Total pions: 10 (50.0%)

Charge Statistics:
Mean charge per track: 12246.96
Min charge: 1743.34
Max charge: 24871.82

Momentum Sta

In [8]:
# Read events from the generated folders
folder_data = read_multi_folder_events(
    folder_names=folders,
    max_files_per_folder=2,  # Read only first 10 files from each folder
    summary_only=False
)


Reading events from 3 folders:

--------------------------------------------------
Folder 1/3: ../datasets/events_1ring
--------------------------------------------------
Found 2 files to analyze

File: event_8.h5
Event Number: 8
Number of tracks: 1
Number of detectors: 9946

Particle Information:
--------------------------------------------------------------------------------
Track # PDG     Q_tot       P_mag (MeV/c)   Direction                Vertex                   
--------------------------------------------------------------------------------
0       Muon    4655.62     458.46          [ 0.35219038  0.56163794 -0.748682  ][ 0.08888973 -0.01390459  0.09131434]

Detector Statistics:
Total charge detected: 4655.62
Mean charge per track: 4655.62
Mean charge per PMT: 0.47
Number of PMTs with signal: 1595

Charge Matrix (Q) - First 10 PMTs:
--------------------------------------------------------------------------------
Track #  PMT-0     PMT-1     PMT-2     PMT-3     PMT-4     PMT-5